In [49]:
# import package 
import time 
import numpy as np
import pandas as pd
import scipy.integrate as integrate

from scipy.stats import norm


In [65]:
def black_sholes(inputs,type_option):
    """
    Calculation of the price of a European option according to the Black-Scholes model
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    r = inputs['r']
    start = time.perf_counter()
    
    d1 = (np.log(x0/k) + (r + 0.5 * sigma**2) * T) / sigma / np.sqrt(T)
    d2 = d1 - sigma * np.sqrt(T)
    
    if type_option == "call":
        price = x0*norm.cdf(d1) - k*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = k*exp(-r*T)*norm.cdf(-d2) - x0*norm.cdf(-d1)
        
    end = time.perf_counter() - start
    
    return price,np.nan,end

In [55]:
def numerical_integration(inputs,type_option,domain):
    """
    Calculation of the price of a European option according to the numerical integration 
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    r = inputs['r']
    start = time.perf_counter()
    
    drift = (r - 0.5 * sigma **2) * T

    # error tolerance (absolute/relative)
    abs_error = inputs["Absolute error"]
    rel_error = inputs["Relative error"]
    
    # integration on R - set of real numbers 
    if domain == 'infinite':
        bound_low = -np.inf
        bound_sup = np.inf
        if type_option == "call":
            function = lambda z : (
                np.exp(-r*T)
                * np.maximum(x0 * np.exp(drift + sigma * z * np.sqrt(T)) - k, 0)
                * norm.pdf(z)
                )
        
        else:
            function = lambda z : (
                np.exp(-r*T)
                * np.maximum(k - x0 * np.exp(drift + sigma * z * np.sqrt(T)), 0)
                * norm.pdf(z)
                )
            
            
    # integration on [0,1]
    else:
        bound_low = 0
        bound_sup = 1
        if type_option == "call":
            # substitution of variable 
            # X ~ N(0,1) ==> U ~ uniform(0,1)
            # U = inverse cumulative distribution function of X
            function = lambda z : (
                np.exp(-r*T)
                * np.maximum(x0 * np.exp(drift + sigma * norm.ppf(z) * np.sqrt(T)) - k, 0)
                )
        
        else:
            function = lambda z : (
                np.exp(-r*T)
                * np.maximum(k - x0 * np.exp(drift + sigma * norm.ppf(z) * np.sqrt(T)), 0)
                )
    
    # computation of a defined integral
    if np.isnan(abs_error) and np.isnan(rel_error):
        integral, uncertainty = integrate.quad(function, bound_low, bound_sup)
    elif not np.isnan(abs_error):
        integral, uncertainty = integrate.quad(function, bound_low, bound_sup, epsabs=abs_error)
    else:
        integral, uncertainty = integrate.quad(function, bound_low, bound_sup, epsrel=rel_error)
        
    end = time.perf_counter() - start

                    
    return integral, uncertainty, end

In [73]:
def monte_carlo_simulation(inputs,type_option):
    """
    Calculation of the price of a European option according to Monte Carlo simulation
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    r = inputs['r']
    N = inputs['N'] # number of paths to generate 
    alpha = inputs['Confidence level']
    seed = inputs['seed']
    
    if not np.isnan(seed):
        np.random.seed((int(seed)))
    
    start = time.perf_counter()
    
    z = np.random.normal(size=int(N))
    
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    
    # terminal value of underlying asset 
    xt = x0 * np.exp(drift + diffusion)
    
    if type_option == "call":
        list_price = np.exp(-r * T) * np.maximum(xt - k, 0)
    else:
        list_price = np.exp(-r * T) * np.maximum(k - xt, 0)
    
    price = np.mean(list_price)
    # unbiased standard deviation, degree of freedom N-1
    std_price = np.std(list_price, ddof=1)
    # margin error 
    delta = norm.ppf(1-(1-alpha)/2) * std_price / np.sqrt(N)
    
    end = time.perf_counter() - start
    
    return float(price), float(delta), end

In [30]:
# enter inputs manually 
def enter_inputs():
    inputs = {}

    inputs['x0'] = float(input("What is the current stock price? "))
    inputs['K'] = float(input("What is the strike price? "))
    inputs['sigma'] = float(input("What is the continuous volatility?"))
    inputs['T'] = float(input("What is the time to maturity? "))
    inputs['r'] = float(input("What is the continuously compounding risk-free interest rate?"))
    
    pricing_method = input("What pricing method do you want to use?")
            
    if pricing_method == "numerical integration":
        # inputs -- numerical integration
        inputs["Absolute error"] = float(input("What is the absolute error tolerance?"))
        inputs["Relative error"] = float(input("What is the relative error tolerance?"))
    if pricing_method == "Monte Carlo simulation":
        # inputs -- Monte Carlo simulation
        inputs['N'] =  int(input("What is the number of paths to generate?"))
        inputs['Confidence level'] = float(input("What is the Confidence level?"))
        inputs['seed'] = input("What is the seed of random generator?")

    return inputs

In [98]:
# inputs
inputs['x0'] = 100
inputs['K'] = 100
inputs['sigma'] = 0.2
inputs['T'] = 1
inputs['r'] = 0.05

# inputs -- numerical integration
inputs["Absolute error"] = np.nan
inputs["Relative error"] = np.nan

# inputs -- Monte Carlo simulation
inputs['N'] = 10000
inputs['Confidence level'] = 0.95
inputs['seed'] = 1

In [99]:
results = {}

results["Call Black-Scholes"] = black_sholes(inputs,"call")

results["Call Integration on R"] = numerical_integration(inputs,"call","infinite")

results["Call Integration on [0,1]"] = numerical_integration(inputs,"call","1")

results["Call Monte Carlo"] = monte_carlo_simulation(inputs,"call")

In [103]:
pd.options.display.float_format = '{:,.8f}'.format
pd.DataFrame(results,index=["value","error","time"])

# Usually, we prefer integrating on [0,1] to integrating on R
# coz the points we choose to integrate are more intense and precise


,Call Black-Scholes,Call Integration on R,"Call Integration on [0,1]",Call Monte Carlo
value,10.45058357,10.45058357,10.45058357,10.55151687
error,nan,0.00000009,0.00000002,0.29191543
time,0.00041708,0.13927943,0.20103613,0.00070133
